In [1]:
import numpy as np
import random
import torch

import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
from torchvision.utils import make_grid, save_image

import json

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# from utils import visualize_cam, Normalize, load_image, plot_gradcam, get_gradcam
# from gradcam import GradCAM, GradCAMpp

from attn_model2 import *

import matplotlib.pyplot as plt

from helpers import *
# from utils import *

from utils2 import *

from copy import deepcopy

from handler import geozoom_handler

In [2]:
image_indices = [0, 427, 775, 1000, 1042]

# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

5 municipalities.


In [3]:
BATCH_SIZE = 1

x_train, y_train, x_val, y_val = train_test_split(np.array(image_names), np.array(y), .9)

train = [(k,v) for k,v in zip(x_train, y_train)]
val = [(k,v) for k,v in zip(x_val, y_val)]

train_dl = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)

for i, o in val_dl:
    print(i[0].split("/")[4])

484020535


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
model = attnNetBinary(in_channels = 512, h = 7, w = 7, batch_size = 1, resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

handler = geozoom_handler(model, device, criterion, optimizer)

In [5]:
# for i in range(50):
while handler.threshold_index > 0:

    for impath, output in train_dl:

        # Prep the input and pass it to the trainer (this could easily be done in one step eventually if ya want)
        input = handler.prep_input(impath)
        handler.train(input, output)
    
    handler.end_epoch(train_dl, val_dl = None)

Epoch:  0
  Training Loss:  1291.7023849487305
Loss thresholds for training:  [0.0, 184.52891213553292, 369.05782427106584, 553.5867364065988, 738.1156485421317, 922.6445606776646, 1107.1734728131976]
Starting from threshold:  6  with value:  1107.1734728131976
  Updating model weights.


Epoch:  1
  Training Loss:  1186.310401916504
  Updating model weights.


Epoch:  2
  Training Loss:  940.1547927856445
  Updating model weights.
  Moving to threshold:  5   |  Next loss benchmark:  922.6445606776646


/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")




Epoch:  3
  Training Loss:  1038.580234527588
  Updating model weights.


Epoch:  4
  Training Loss:  807.3309936523438
  Updating model weights.
  Moving to threshold:  4   |  Next loss benchmark:  738.1156485421317


Epoch:  5
  Training Loss:  1087.2178039550781
  Updating model weights.


Epoch:  6
  Training Loss:  902.3010330200195
  Updating model weights.


Epoch:  7
  Training Loss:  1092.048210144043


Epoch:  8
  Training Loss:  823.0031280517578
  Updating model weights.


Epoch:  9
  Training Loss:  745.8505859375
  Updating model weights.


Epoch:  10
  Training Loss:  865.6685791015625


Epoch:  11
  Training Loss:  735.8683471679688
  Updating model weights.
  Moving to threshold:  3   |  Next loss benchmark:  553.5867364065988


Epoch:  12
  Training Loss:  829.1408538818359
  Updating model weights.


Epoch:  13
  Training Loss:  848.6707534790039


Epoch:  14
  Training Loss:  837.9545364379883


Epoch:  15
  Training Loss:  758.9454116821289
  Updating model weigh

In [8]:
handler.image_sizes

{'484002003': (0, 224, 0, 224),
 '484016045': (0, 224, 0, 224),
 '484020489': (0, 224, 0, 224),
 '484020254': (0, 224, 0, 224)}

In [9]:
handler.threshold_weights.keys()

dict_keys([6, 5, 4, 3, 2, 1])

In [10]:
for i, o in val_dl:
    print(i, o)

('../pooling/data/MEX2/484020535/pngs/484020535_2010_1_box484020535_MAY.png',) tensor([89.], dtype=torch.float64)


In [39]:
def run_validation(model, input, weights_dict):
    
    muni_id = input[0].split("/")[4]
    cur_image = load_inputs(input[0])
    
    print("starting shape: ", cur_image.shape)
    
    
    for k in weights_dict.keys():
        
        model.load_state_dict(weights_dict[k])
        model.eval()
        IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
        gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda()) 
        cur_image, new_dims = handler.clip_input(cur_image, attn_heatmap)
        print(cur_image.shape)
        plot_gradcam(gradcam)
        plt.savefig(f"threshold{k}_muni{muni_id}.png")
        plt.clf()

        if k == 1:
            
            return model(cur_image.cuda()).item()
        
        
for i, o in train_dl:

    print(round(run_validation(model, i, handler.threshold_weights), 2), "     ", o.item())
    
#     print(o)

starting shape:  torch.Size([1, 3, 643, 457])
torch.Size([1, 3, 450, 319])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
175.85       334.0
starting shape:  torch.Size([1, 3, 423, 628])
torch.Size([1, 3, 296, 439])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
142.18       1352.0
starting shape:  torch.Size([1, 3, 562, 981])
torch.Size([1, 3, 393, 686])
torch.Size([1, 3, 275, 480])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
2505666.0       348.0
starting shape:  torch.Size([1, 3, 1488, 3072])
torch.Size([1, 3, 1041, 2150])
torch.Size([1, 3, 728, 1505])
torch.Size([1, 3, 509, 1053])
torch.Size([1, 3, 356, 737])
torch.Size([1, 3, 249, 515])
torch.Size([1, 3, 224, 224])
56354.76       3162.0


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

...

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>